In [1]:
import numpy as np

import rospy
import actionlib
from omo_simple_position_controller.msg import VanillaPositionAction, VanillaPositionGoal
from ar_track_alvar_msgs.msg import AlvarMarkers

from tf.transformations import euler_from_quaternion

rospy.init_node('marker_tracker')

In [2]:
target_marker = {
    'id' : -1,
    'x_pos' : 0.,
    'y_pos' : 0.,
    'direction' : 0.,
    'x_slope' : 1.1,
    'x_bias' : 0.,
    'y_slope' : 1.,
    'y_bias' : 0.,
}

next_marker = {
    'id' : -1,
    'x_pos' : 0.,
    'y_pos' : 0.,
    'direction' : 0.,
}

In [3]:
def sub_markers_callback(msg):
    global target_marker, next_marker
    
    marker_id = []
    length = []

    x_pos = []
    y_pos = []
    z_pos = []
    ori_x = []
    ori_y = []
    ori_z = []
    
    for each in msg.markers:
        marker_id.append(each.id)
        
        x = each.pose.pose.position.x
        y = each.pose.pose.position.y
        z = each.pose.pose.position.z

        x_pos.append(x)
        y_pos.append(y)
        z_pos.append(z)
        
        length.append(np.sqrt(x**2 + y**2 + z**2))

        r, p, th = euler_from_quaternion([each.pose.pose.orientation.x, 
                                          each.pose.pose.orientation.y, 
                                          each.pose.pose.orientation.z, 
                                          each.pose.pose.orientation.w])

        ori_x.append(r)
        ori_y.append(p)
        ori_z.append(th)
        
    if len(length)>0:
        idx_sort = np.argpartition(np.array(length), 0)
        idx = np.argmin(idx_sort)
        target_marker['id'] = marker_id[idx]
        target_marker['x_pos'] = target_marker['x_slope'] * x_pos[idx] + target_marker['x_bias']
        target_marker['y_pos'] = target_marker['y_slope'] * y_pos[idx] + target_marker['y_bias']
        
        if len(idx_sort)>1:
            next_idx = int(np.where(idx_sort==1)[0])
            next_marker['id'] = marker_id[next_idx]
            next_marker['x_pos'] = target_marker['x_slope'] * x_pos[next_idx] + target_marker['x_bias']
            next_marker['y_pos'] = target_marker['y_slope'] * y_pos[next_idx] + target_marker['y_bias']
            
        else:
            next_marker['id'] = -1
        
    else:
        target_marker['id'] = -1
        

sub_markers = rospy.Subscriber('ar_pose_marker', AlvarMarkers, sub_markers_callback)

In [4]:
target_marker

{'direction': 0.0,
 'id': -1,
 'x_bias': 0.0,
 'x_pos': 0.0,
 'x_slope': 1.1,
 'y_bias': 0.0,
 'y_pos': 0.0,
 'y_slope': 1.0}

In [5]:
next_marker

{'direction': 0.0, 'id': -1, 'x_pos': 0.0, 'y_pos': 0.0}

In [6]:
from geometry_msgs.msg import Twist

pub = rospy.Publisher("/cmd_vel", Twist, queue_size = 1)

In [7]:
def move_robot(x, y, th):
    from omo_simple_position_controller.msg import VanillaPositionAction, VanillaPositionGoal
    
    goal_pos = VanillaPositionGoal()
    goal_pos.x = x
    goal_pos.y = y
    goal_pos.theta = th
    
    client = actionlib.SimpleActionClient('vanilla_position_controller', VanillaPositionAction)
    
    client.send_goal(goal_pos)

In [8]:
is_target_0 = bool(target_marker['id'] == 0)
is_target_1 = bool(target_marker['id'] == 1)
is_target_2 = bool(target_marker['id'] == 2)
is_next_target_0 = bool(next_marker['id'] == 0)
is_next_target_1 = bool(next_marker['id'] == 1)
is_next_target_2 = bool(next_marker['id'] == 2)

In [17]:
is_target_0, is_next_target_1, is_target_1, is_next_target_0

(True, False, False, False)

In [ ]:
sub_markers = rospy.Subscriber('ar_pose_marker', AlvarMarkers, sub_markers_callback)

while True:
    process = False
    if target_marker['id'] == 0:
        process = True
        while process:
            print "== move 0"
            print target_marker['x_pos'], target_marker['y_pos'], 0.
            #move_robot(target_marker['x_pos'], target_marker['y_pos'], 0.)
        
    if target_marker['id'] == 1:
        process = True
        while process:
            print "== move 1"
            print target_marker['x_pos'], target_marker['y_pos'], 0.
            #move_robot(target_marker['x_pos'], target_marker['y_pos'], 0.)
    
    
    
#     if is_target_0 & is_next_target_1:
#         while is_target_0:
#             print "== move 01"
#             #move_robot(target_marker['x_pos'], target_marker['y_pos'], 0.)
            
#     if is_target_1 & is_next_target_0:
#         while is_target_1:
#             print "== move 10"
            #move_robot(target_marker['x_pos'], target_marker['y_pos'], 0.)
    
    
#     if (is_target_0 & is_next_target_1) | (is_target_1 & is_next_target_0):
#         trace_marker = target_marke['id']
#         while target_marker['x_pos'] > 0.5:
#             move_robot(target_marker['x_pos'], target_marker['y_pos'], 0.)

In [19]:
target_marker

{'direction': 0.0,
 'id': 1,
 'x_bias': 0.0,
 'x_pos': 0.8807476027238079,
 'x_slope': 1.1,
 'y_bias': 0.0,
 'y_pos': 0.08107104483601511,
 'y_slope': 1.0}

In [20]:
move_robot(target_marker['x_pos'], target_marker['y_pos'], 0.)